In [1]:
import wget
import gzip
import sys
import os
import platform
import pandas
import json
import pickle
import multiprocessing as mp
from collections import OrderedDict, defaultdict
import redis

In [10]:
fams_dict = json.load(open("EAGLEdb_0/bacteria/bact_fam.json"))
genera_num_list = list()
sp_num_list = list()
strain_num_list = list()
special_keys = ("16S_rRNA_tree", "16S_rRNA_tsv", "16S_rRNA_fasta", "blastdb", "repr_profile")
fam_num = 0
for fam_name in fams_dict.keys():
    if fam_name in ("db_dir", "db_repr_profiles"):
        continue
    fam_num += 1
    genera_num = 0
    sp_num = 0
    strain_num = 0
    for genus in fams_dict[fam_name].keys(): 
        if genus in special_keys:
            continue
        genera_num += 1
        for sp in fams_dict[fam_name][genus].keys():
            sp_num += 1
            for strain in fams_dict[fam_name][genus][sp].keys():
                strain_num += 1
    genera_num_list.append(genera_num)
    sp_num_list.append(sp_num)
    strain_num_list.append(strain_num)

print(fam_num)
print(sum(genera_num_list))
print(sum(sp_num_list))    
print(sum(strain_num_list))

337
749
1387
1406


In [13]:
def list_to_txt(in_list, txt_path):
    f = open(txt_path, 'w')
    for elm in in_list:
        f.write(str(elm)+"\n")
    f.close()
    

list_to_txt(strain_num_list, "strain_num.txt")

In [6]:
fams_dict["db_repr_profiles"]

u'EAGLEdb/bacteria/db_repr_profiles'

In [2]:
from EAGLE.eag_location_explorer import get_btax

#in_fasta = "EAGLEdb/bacteria_test/GCF_002005165.1_ASM200516v1_genomic.fna"
in_fasta = "EAGLEdb/bacteria_test/Acetoanaerobium_sticklandii_16S_rRNA.fasta"
db_json = "EAGLEdb/bacteria/bact_fam.json"
db_info = json.load(open(db_json))
get_btax(in_fasta,
         db_info["db_repr_profiles"],
         btax_names=db_info.keys(),
         hmmer_inst_dir="/home/loven-doo/tools/bin",
         remove_scan_out=False)

2018-05-23 00:02:19,148 - INFO - hmmscan started
2018-05-23 00:02:19,153 - INFO - hmmscan finished


{'EAGLEdb/bacteria_test/Acetoanaerobium_sticklandii_16S_rRNA': u'Peptostreptococcaceae'}

In [219]:
redis_conn = redis.StrictRedis()
redis_conn.llen("bacteria_queue")

ConnectionError: Error 111 connecting to localhost:6379. Connection refused.

In [107]:
def load_phylip_dist_matrix(matrix_path):
    matr_f = open(matrix_path)
    lines_dict = OrderedDict()
    seqs_list = list()
    for line_ in matr_f:
        line = None
        line = line_.strip()
        if not line:
            continue
        line_list = filter_list(line.split())
        if len(line_list) == 1:
            continue
        seqs_list.append(line_list[0])
        lines_dict[line_list[0]] = line_list[1:]
    matrix = pandas.DataFrame.from_dict(data=lines_dict, orient='index')
    matrix.columns = seqs_list
    return matrix


def filter_list(in_list):
    filtered_list = list()
    for elm_ in in_list:
        elm = None
        elm = elm_.strip()
        if elm:
            filtered_list.append(elm)
    return filtered_list
    
    
matr_f_path = 'dist_matr.ph'
dist_matrix = load_phylip_dist_matrix(matr_f_path)
print sum(map(float, list(dist_matrix.loc['Seq1abcdef'])))
print(dist_matrix.index)

0.379063
Index([u'Seq1abcdef', u'Seq2abcd', u'Seq3abcdef'], dtype='object')


In [207]:
bacteria_json_f = open("EAGLEdb/bacteria/bacteria.json")
bacteria_info_list = json.load(bacteria_json_f)
bacteria_json_f.close()
analyzed_bacteria = mp.Manager().dict()
for bacterium_info in bacteria_info_list:
    if os.path.exists(bacterium_info[u'16S_rRNA_file']):
        analyzed_bacteria[bacterium_info[u'strain'].replace("_", " ")] = True
analyzed_bacteria_f = open("analyzed_bacteria.p", 'wb')
pickle.dump(analyzed_bacteria, analyzed_bacteria_f)
analyzed_bacteria_f.close()

In [213]:
analyzed_bacteria["Buchnera_aphidicola_str._Bp_(Baizongia_pistaciae)".replace("_", " ")]

True

In [119]:
test_dict = {"k1": 2, "k2": 1, "k3": 3}
sorted(test_dict.items(), key=lambda x: x[1])

[('k2', 1), ('k1', 2), ('k3', 3)]

In [132]:
test_dict = {"set1": {"k1": 2, "k2": 1, "k3": 3}, 
             "set2": {"k4": 3, "k5": 6, "k6": 2}}
for key in test_dict.keys():
    test_dict[key] = sorted(test_dict[key].items(), key=lambda x: x[1])
print test_dict
falsh_dict = {"k1": "a", "k2": "b", "k3": "c", "k4": "d", "k5": "e", "k6": "f"}
trans_dict = {"k1": "set1", "k2": "set1", "k3": "set1", "k4": "set2", "k5": "set2", "k6": "set2"}
filter(lambda x: x[0] == test_dict[trans_dict[x[0]]][0][0], falsh_dict.items())

{'set1': [('k2', 1), ('k1', 2), ('k3', 3)], 'set2': [('k6', 2), ('k4', 3), ('k5', 6)]}


[('k2', 'b'), ('k6', 'f')]

In [141]:
print ((None, None), (None, None))

((None, None), (None, None))


In [136]:
pd_dict = {"Key1": {"A": 1, "B": 2},
           "Key2": {"A": 3, "B": 4}}
pandas.DataFrame.from_dict(data=pd_dict, orient="index")

,A,B
Key1,1,2
Key2,3,4


In [159]:
d = {"k1": 1, "k2": 2}
d = defaultdict(int)
d["k1"] = 1
d["k2"] = 2
print(d["A"])
if d.pop("k1") == 1:
    print 1
a = d.pop("A")
print k1, a, d.keys()

0
1
1 0 ['k2']


In [168]:
def get_tree_from_dict(input_dict, stop_level=2):
    tree = dict()
    if stop_level == 1:
        return input_dict.keys()
    for key in input_dict.keys():
        tree[key] = get_tree_from_dict(input_dict[key], stop_level=stop_level-1)
    return tree

d = {"k1": {"k3": {"k5": 5,
                   "k6": 6},
            "k4": {"k7": 7,
                   "k8": 8}},
     "k2": {"k9": {"k11": 11,
                   "k12": 12},
            "k10": {"k13": 13,
                    "k14": 14}}}
get_tree_from_dict(d, 3)

{'k1': {'k3': ['k6', 'k5'], 'k4': ['k8', 'k7']},
 'k2': {'k10': ['k13', 'k14'], 'k9': ['k12', 'k11']}}

In [177]:
dicts_list = [{"col1": 1, "col2": 2, "col3": 3},
              {"col1": 4, "col2": 5, "col3": 6},
              {"col1": 7, "col2": 8, "col3": 9}]
appl_df = pandas.DataFrame(dicts_list)
print(appl_df)
list(appl_df.apply(lambda x: (x["col1"], x["col2"]) if x["col3"] > 5 else None, axis=1))

   col1  col2  col3
0     1     2     3
1     4     5     6
2     7     8     9


[None, (4, 5), (7, 8)]

In [92]:
def dump_phylip_dist_matrix(dist_matrix, matrix_path):
    matr_f = open(matrix_path, 'w')
    matr_f.write("    %s\n" % len(dist_matrix.columns))
    for seq in dist_matrix.index:
        num_spaces_to_add = 10 - len(seq)
        spaces_to_add = [" " for i in range(num_spaces_to_add)]
        matr_f.write("%s %s\n" % (seq+"".join(spaces_to_add), " ".join(dist_matrix.loc[seq].tolist())))
    matr_f.close()
    

dump_phylip_dist_matrix(dist_matrix, "dumped_matrix.ph")

In [ ]:
def reduce_seq_names(fasta_dict, num_letters=10, num_words=4):
    if num_letters < 6:
        print("Number of letters must be at least 6")
        return 1
    if num_words < 2:
        print("Number of words must be at least 2")
        return 1
    splitters_repl = {"\t": " ",
                      ",": " ",
                      ";": " ",
                      ".": " ",
                      ":": " ",
                      "|": " ",
                      "/": " ",
                      "\\": " "}
    parts_size_list = get_part_size_list(num_letters, num_words)
    reduced_fasta_dict = dict()
    seq_names_dict = dict()
    for seq_name in fasta_dict.keys():
        reduced_seq_name = None
        seq_name_list = filter_list("".join([splitters_repl.get(s, s) for s in seq_name]).split())
        parts = list()
        for i in range(num_words):
            try:
                parts.append(seq_name_list[i][:parts_size_list[i]])
            except IndexError:
                break
        reduced_seq_name = "".join(parts)
        res_len = num_letters - len(reduced_seq_name)
        un_num = 1
        un_fix = get_un_fix(un_num, res_len)
        while seq_names_dict.get(reduced_seq_name+un_fix, None):
            un_fix = None
            un_num += 1
            un_fix = get_un_fix(un_num, res_len)
        reduced_fasta_dict[reduced_seq_name+un_fix] = fasta_dict[seq_name]
        seq_names_dict[reduced_seq_name+un_fix] = seq_name 
    return reduced_fasta_dict, seq_names_dict


def get_part_size_list(num_letters, num_words):
    if num_letters == 6: 
        return [2, 3]
    if num_letters == 7:
        if num_words >= 3:
            return [2, 3, 1]
        else:
            return [2, 3]
    if num_letters == 8:
        if num_words >= 4:
            return [2, 3, 1, 1]
        elif num_words == 3:
            return [3, 3, 1]
        else:
            return [3, 3]
    if num_letters == 9:
        if num_words >= 4:
            return [3, 3, 1, 1]
        elif num_words == 3:
            return [3, 3, 1]
        else:
            return [3, 4]
    if num_letters == 10:
        if num_words >= 4:
            return [3, 4, 1, 1]
        elif num_words == 3:
            return [3, 4, 1]
        else:
            return [4, 4]
    if num_letters >= 11:
        if num_words >= 4:
            return [3, 4, 1, 1]
        elif num_words == 3:
            return [4, 4, 1]
        else:
            return [4, 5]


def get_un_fix(un_num, fix_len):
    un_codes = ["_", '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E']
    # 'N' - undefined (num duplicates is bigger than len(un_codes))
    if fix_len == 1:
        try:
            return un_codes[un_num-1]
        except IndexError:
            return 'N'
    elif fix_len == 0:
        return ""
    elif un_num <= len(un_codes):
        return un_codes[0] + get_un_fix(un_num, fix_len-1)
    else:
        filled_rank = len(un_codes)**(fix_len-1)
        return un_codes[un_num//filled_rank - 1] + get_un_fix(un_num%filled_rank, fix_len-1)
    

fasta_dict = {"seq1abc_defghi jkhhg": "ACGTTGCGGTAA", 
              "seq2hgft asdad.fdggjh fgjgh fgjhd": "GGTTAAG",
              "seq3hgft asdad fdggjh\fgjgh fkn": "GGTTAAG"}
reduce_seq_names(fasta_dict)

In [182]:
def prepare_summary_tables(summary_table=None, out_table_path=None):
    if not summary_table and not out_table_path:
        try:
            summary_table = sys.argv[1]
            out_table_path = sys.argv[2]
        except IndexError:
            print("Number of arguments must be 2: 1 - summary table path; 2 - output table path")
    summary_df = pandas.read_csv(summary_table, header=1, sep="\t", dtype=str)
    lines_list = summary_df.apply(lambda df_row: {
        "org_name": df_row['organism_name'],
        "ncbi_link": df_row['ftp_path'],
        "repr": True if "repr" in df_row['refseq_category'].lower() else False,
        } if df_row['assembly_level'].lower() == "complete genome" else None, axis=1)
    prepared_df = pandas.DataFrame(filter(None, lines_list))
    prepared_df = prepared_df[["org_name", "ncbi_link", "repr"]]
    prepared_df.to_csv(out_table_path, sep="\t", index=False)

prepare_summary_tables(summary_table="assembly_summary_genbank.txt", out_table_path="genbank_bacteria_table.txt")
prep_df = pandas.read_csv("genbank_bacteria_table.txt", sep="\t", dtype=str)
prep_df.head(20)

,org_name,ncbi_link,repr
0,Azorhizobium caulinodans ORS 571,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,True
1,Buchnera aphidicola str. Sg (Schizaphis graminum),ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,True
2,Buchnera aphidicola str. Bp (Baizongia pistaciae),ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,True
3,Buchnera aphidicola str. APS (Acyrthosiphon pi...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,False
4,Buchnera aphidicola str. Tuc7 (Acyrthosiphon p...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,False
5,Buchnera aphidicola str. 5A (Acyrthosiphon pisum),ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,False
6,Buchnera aphidicola BCc,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,True
7,Buchnera aphidicola str. LL01 (Acyrthosiphon p...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,False
8,Buchnera aphidicola str. TLW03 (Acyrthosiphon ...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,False
9,Buchnera aphidicola str. JF99 (Acyrthosiphon p...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,False


In [183]:
!head -n 21 genbank_bacteria_table.txt

org_name	ncbi_link	repr
Azorhizobium caulinodans ORS 571	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/010/525/GCA_000010525.1_ASM1052v1	True
Buchnera aphidicola str. Sg (Schizaphis graminum)	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/007/365/GCA_000007365.1_ASM736v1	True
Buchnera aphidicola str. Bp (Baizongia pistaciae)	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/007/725/GCA_000007725.1_ASM772v1	True
Buchnera aphidicola str. APS (Acyrthosiphon pisum)	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/009/605/GCA_000009605.1_ASM960v1	False
Buchnera aphidicola str. Tuc7 (Acyrthosiphon pisum)	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/021/065/GCA_000021065.1_ASM2106v1	False
Buchnera aphidicola str. 5A (Acyrthosiphon pisum)	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/021/085/GCA_000021085.1_ASM2108v1	False
Buchnera aphidicola BCc	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/090/965/GCA_000090965.1_ASM9096v1	True
Buchnera aphidicola str. LL01 (Acyrthosiphon pisum)	ftp://ftp.nc

In [199]:
print(type(prep_df.loc[4]["repr"]))
print(prep_df.shape[0])

<type 'str'>
10144


In [93]:
for i in range(-5):
    print(i)

In [10]:
try:
    wget.download('ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Abiotrophia_defectiva/representative/GCF_000160075.2_ASM16007v2/GCF_000160075.2_ASM16007v2_rna_from_genomic.fna.gz', out="EAGLEdb")
except IOError:
    print "That's it."

In [11]:
f = gzip.open('EAGLEdb/GCF_000160075.2_ASM16007v2_rna_from_genomic.fna.gz', 'rb')
for line in f:
    print line

>lcl|NZ_KI535340.1_trna_1 [locus_tag=GCWU000182_RS00760] [product=tRNA-Leu] [location=157701..157786] [gbkey=tRNA]

GCGGAGATGGCGGAACGGCAGACGCGCATGCTTCAGGCGCATGTGTCCAATAGGACGTGAGGGTTCAAATCCCTTTCTCC

GCATCA

>lcl|NZ_KI535340.1_tmrna_2 [gene=ssrA] [locus_tag=GCWU000182_RS09735] [db_xref=RFAM:RF00023] [product=transfer-messenger RNA] [location=191584..191947] [gbkey=tmRNA]

GGGGATGTCACGGTTTCGACAGGTATTGTTCGAGCTTTTGATGCGTCAGGAGGGTTGCGTCCGCCTTAAAACGCTACAGT

TAAATATAACTGCAAACAACCAAAACAACTCTTTCGCTTTAGCAGCTTAATAACCTGCAGCGGAAAATCAGCCTAACATC

GCCCATGTGCTAGTAGCTGGTTTTATTTAAGTGGGATACGCGTGATTTATCCGTCTGTTAAATCACGAAGAGATAATCAG

ACTCGCAAGGATGACAGCCCGTCTAGTGGCGGTCTCTGCGCTAAATCAAATAGCTAGACTATTGACGTAGACTCGATTGT

GGCGAGATACTTGGACAGGGGTTCGATTCCCCTCATCTCCATAA

>lcl|NZ_KI535340.1_ncrna_3 [gene=rnpB] [locus_tag=GCWU000182_RS09740] [db_xref=RFAM:RF00011] [product=RNase P RNA component class B] [ncRNA_class=RNase_P_RNA] [location=complement(193281..193674)] [gbkey=ncRNA]

GATTTTAGGTAATCGCGGCTAGCGATAGTTGAGGAAAGTCC

In [17]:
platform.system()

'Linux'

In [215]:
class MyClass:
     def __init__(self):
        self.a = 5
        

my_class = MyClass()
print(my_class.a)
my_class.my_method()

5


AttributeError: MyClass instance has no attribute 'my_method'

In [217]:
mplist = mp.Manager().list()
if mplist:
    print("Full 1")
mplist.append(1)
if mplist:
    print("Full 2")

Full 2


In [227]:
from EAGLEdb.lib.db_creator import create_profiles_db


families_dict = json.load(open("EAGLEdb/bacteria_test/bact_fam.json"))
create_profiles_db(families_dict, "EAGLEdb/bacteria_test", hmmer_inst_dir="/home/loven-doo/tools/bin")

In [231]:
!/home/loven-doo/tools/bin/hmmscan --cpu 6 EAGLEdb/bacteria_test/db_repr_profiles EAGLEdb/bacteria_test/GCF_002005165.1_ASM200516v1_genomic.fna

# hmmscan :: search sequence(s) against a profile database
# HMMER 3.1b2 (February 2015); http://hmmer.org/
# Copyright (C) 2015 Howard Hughes Medical Institute.
# Freely distributed under the GNU General Public License (GPLv3).
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query sequence file:             EAGLEdb/bacteria_test/GCF_002005165.1_ASM200516v1_genomic.fna
# target HMM database:             EAGLEdb/bacteria_test/db_repr_profiles
# number of worker threads:        6
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       NZ_CP019699.1  [L=3629225]
Description: Novibacillus thermophilus strain SG-1, complete genome
^C


In [7]:
os.path.exists("EAGLEdb/bacteria/bact_fam.json")

True